# Hugginface Pipeline (Nicht Teil der Eval)

In [14]:
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"



df = pd.read_csv("drive_download/summarization_data_newsroom.csv")
df = df.drop(columns=['density_bin'])

traindf, testdf = train_test_split(df, test_size=0.2)

train = Dataset.from_pandas(traindf)
test = Dataset.from_pandas(testdf)


dataset = DatasetDict()

dataset['train'] = train
dataset['test'] = test

In [15]:
from transformers import AutoTokenizer

checkpoint = "facebook/bart-large-cnn"#t5-small" #model
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Could not locate the tokenizer configuration file, will try to use the model config instead.


Downloading:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

loading configuration file config.json from cache at /home/rechcons/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/3d224934c6541b2b9147e023c2f6f6fe49bd27e1/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointi

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

loading file vocab.json from cache at /home/rechcons/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/3d224934c6541b2b9147e023c2f6f6fe49bd27e1/vocab.json
loading file merges.txt from cache at /home/rechcons/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/3d224934c6541b2b9147e023c2f6f6fe49bd27e1/merges.txt
loading file tokenizer.json from cache at /home/rechcons/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/3d224934c6541b2b9147e023c2f6f6fe49bd27e1/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /home/rechcons/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/3d224934c6541b2b9147e023c2f6f6fe49bd27e1/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activati

In [16]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/22 [00:00<?, ?ba/s]

In [18]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [19]:
import evaluate

rouge = evaluate.load("rouge")

In [20]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [21]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to(device)

loading configuration file config.json from cache at /home/rechcons/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/3d224934c6541b2b9147e023c2f6f6fe49bd27e1/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-large-cnn",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 4096,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 4096,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 12,
  "eos_token_id": 2,
  "force_bos_token_to_be_generated": true,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointi

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /home/rechcons/.cache/huggingface/hub/models--facebook--bart-large-cnn/snapshots/3d224934c6541b2b9147e023c2f6f6fe49bd27e1/pytorch_model.bin
All model checkpoint weights were used when initializing BartForConditionalGeneration.

All the weights of BartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-cnn.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BartForConditionalGeneration for predictions without further training.


In [1]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

NameError: name 'Seq2SeqTrainingArguments' is not defined

In [ ]:
#trainer.save_model("zsmf_trans")

Saving model checkpoint to zsmf_trans
Configuration saved in zsmf_trans/config.json
Model weights saved in zsmf_trans/pytorch_model.bin
tokenizer config file saved in zsmf_trans/tokenizer_config.json
Special tokens file saved in zsmf_trans/special_tokens_map.json
Copy vocab file to zsmf_trans/spiece.model
